Initialize the cloud loader

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import openfoamcloud.providers.google.google_provider as google_provider
cloud_provider = google_provider.GoogleCloud()
cloud_provider.initialize()

Specify base case path

In [ ]:
base_case_path = "./base/case_1"
base_case_path = "/home/vinzenz/Downloads/Case_und_Runscript/OF5_DOE_CarRim_VolFracSpreading_@volt@_@rhop@_@dia_mean_p@_VSolverOnly"

Create Project

In [ ]:
project_name = "test_project_1"
study_name = "study_1"
cloud_provider.create_project(project_name, base_case_path)

Create parameter study

In [ ]:
from math import log10, floor
from openfoamcloud.structures.case import InpurtParameter

startValue=0.000001#1000 #0.00000500#10000
endValue=0.000070 #0.00020500#60000
steps=2

def round_sig(x, sig=3):
  return round(x, sig-int(floor(log10(abs(x))))-1)

valueList=[]
value=startValue
inc=(endValue-startValue)/steps

for i in range(steps+1):
  value=format(round_sig(startValue+inc*i,4),'.2E') 
  valueList.append(str(value))


input_parameters_dicts = [
        { "path": "0/include/initialConditions","variable_name": "volt","values": ["50000"]},
        { "path": "constant/particleProperties","variable_name": "rhop","values": ["2300"]},
        { "path": "constant/particleProperties","variable_name": "dia_mean_p","values": ["0.000075"]},
        { "path": "constant/particleProperties","variable_name": "dev_dia_p","values":valueList}
      ]

input_parameters = [InpurtParameter(**i) for i in input_parameters_dicts]
run_script =  "cd ./Solver && wclean && wmake && cd .. && MyEstaticChargeDensityDivFoam5_volfrac_homog_V4_1 && pvpython volFrac_Coating_CarRim_V1.py"

cloud_provider.create_parameter_stduy(project_name, study_name, run_script, input_parameters)

Show Case Names

In [ ]:
case_names = cloud_provider.get_case_names(project_name, study_name)
print(case_names)

Run specific case

In [ ]:
cloud_provider.run_case(project_name, study_name, case_names[1], file_logging=False)

In [ ]:
jobs = cloud_provider.get_active_runs()
print(jobs)

In [ ]:
import time

running_jobs = len(cloud_provider.get_active_runs_by_parameter_study(project_name, study_name))
while running_jobs > 0:
    running_jobs = len(cloud_provider.get_active_runs_by_parameter_study(project_name, study_name))
    print(f"{running_jobs} jobs running.")
    time.sleep(10)
